# 0. 기본 설정 및 라이브러리 불러오기

In [6]:
# Python 버전 확인
!python --version

# 설치된 패키지 목록을 requirements.txt로 저장
!pip list --format=freeze > requirements.txt

# requirements.txt 내용 확인
!cat requirements.txt

Python 3.10.12
absl-py==1.4.0
accelerate==0.34.2
aiohappyeyeballs==2.4.3
aiohttp==3.10.10
aiosignal==1.3.1
alabaster==0.7.16
albucore==0.0.19
albumentations==1.4.20
altair==4.2.2
annotated-types==0.7.0
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.20.0
astropy==6.1.4
astropy-iers-data==0.2024.10.21.0.33.21
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==24.2.0
audioread==3.0.1
autograd==1.7.0
babel==2.16.0
backcall==0.2.0
beautifulsoup4==4.12.3
bigframes==1.24.0
bigquery-magics==0.4.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.4.3
Bottleneck==1.4.2
bqplot==0.12.43
branca==0.8.0
CacheControl==0.14.0
cachetools==5.5.0
catalogue==2.0.10
certifi==2024.8.30
cffi==1.17.1
chardet==5.2.0
charset-normalizer==3.4.0
chex==0.1.87
clarabel==0.9.0
click==8.1.7
cloudpathlib==0.20.0
cloudpickle==3.1.0
cmake==3.30.5
cmdstanpy==1.2.4
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.5
cons==0.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
print(f"GPU 사용 가능 여부: {torch.cuda.is_available()}")

GPU 사용 가능 여부: True


In [ ]:
#이미지 데이터 수 확인
import os
# 이미지 데이터 폴더 경로 설정
training_image_path = "/content/drive/MyDrive/final/dataset/training_image"
validation_image_path = "/content/drive/MyDrive/final/dataset/validation_image"

# 폴더 내 파일 목록 확인 + 숫자 확인
training_image_num = len(os.listdir(training_image_path)) #os.listdir : 폴더 내의 모든 파일과 디렉토리를 리스트로 반환
validation_image_num = len(os.listdir(validation_image_path))

print(f"Number of training images: {training_image_num}")
print(f"Number of validation images: {validation_image_num}")

Number of training images: 4070
Number of validation images: 951


# 1. 스타일 분류 기준표 생성

In [ ]:
import os
import pandas as pd
from collections import defaultdict

In [ ]:
# 파일명을 분석하여 성별 & 스타일 정보를 추출하는 함수
def extract_gender_style_info(folder_path):
    # 성별과 스타일별 이미지 수를 저장할 딕셔너리
    gender_style_count = defaultdict(int)

    # 폴더 내 모든 파일명 가져오기
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.jpg'):  # jpg 파일만 처리
            # 파일명 분해: 예시 - "W_96469_60_minimal_W.jpg"
            parts = file_name.split('_')
            if len(parts) >= 5:
                gender = '여성' if parts[-1][0] == 'W' else '남성'  # 성별 추출
                style = parts[-2]  # 스타일 추출
                # 성별 & 스타일 기준으로 이미지 수 카운트
                gender_style_count[(gender, style)] += 1

    return gender_style_count

# 각 데이터 폴더에서 성별 & 스타일별 통계 정보 추출
training_stats = extract_gender_style_info(training_image_path)
validation_stats = extract_gender_style_info(validation_image_path)

# 통계 정보를 데이터프레임으로 변환
def convert_to_dataframe(stats_dict):
    # 딕셔너리를 데이터프레임으로 변환 후, '성별', '스타일', '이미지 수'로 분리
    df = pd.DataFrame(list(stats_dict.items()), columns=['성별 & 스타일', '이미지 수'])
    df[['성별', '스타일']] = pd.DataFrame(df['성별 & 스타일'].tolist(), index=df.index)
    df.drop(columns='성별 & 스타일', inplace=True)

    # 컬럼 순서를 성별, 스타일, 이미지 수 순으로 변경
    df = df[['성별', '스타일', '이미지 수']]

    # 성별, 스타일 기준으로 정렬 (여성 -> 남성, a -> z)
    df = df.sort_values(by=['성별', '스타일'], ascending=[False, True])

    # 인덱스 초기화하여 순서 번호 재정렬
    df.reset_index(drop=True, inplace=True)
    return df

In [ ]:
# Training 데이터 통계표
training_df = convert_to_dataframe(training_stats)
print("Training 데이터 통계표:")
print(training_df)

# Validation 데이터 통계표
validation_df = convert_to_dataframe(validation_stats)
print("\nValidation 데이터 통계표:")
print(validation_df)

Training 데이터 통계표:
    성별             스타일  이미지 수
0   여성      athleisure     67
1   여성   bodyconscious     95
2   여성        cityglam     67
3   여성         classic     77
4   여성           disco     37
5   여성         ecology     64
6   여성        feminine    154
7   여성      genderless     77
8   여성          grunge     31
9   여성          hiphop     48
10  여성          hippie     91
11  여성          kitsch     91
12  여성        lingerie     55
13  여성          lounge     45
14  여성        military     33
15  여성         minimal    139
16  여성        normcore    153
17  여성        oriental     78
18  여성          popart     41
19  여성       powersuit    120
20  여성            punk     65
21  여성           space     37
22  여성  sportivecasual    157
23  남성            bold    268
24  남성          hiphop    274
25  남성          hippie    260
26  남성             ivy    237
27  남성     metrosexual    278
28  남성            mods    269
29  남성        normcore    364
30  남성  sportivecasual    298

Validation 데이터 통계표:
 

In [ ]:
# 여성 스타일 목록
female_styles = set(training_df[training_df['성별'] == '여성']['스타일'].unique())

# 남성 스타일 목록
male_styles = set(training_df[training_df['성별'] == '남성']['스타일'].unique())

# 중복 스타일 (양쪽 모두 존재)
common_styles = female_styles & male_styles

# 전체 고유 스타일 수
total_styles = female_styles | male_styles

print(f"전체 고유 스타일 수: {len(total_styles)}개")
print(f"여성 전용 스타일: {len(female_styles - common_styles)}개")
print(f"남성 전용 스타일: {len(male_styles - common_styles)}개")
print(f"공통 스타일: {len(common_styles)}개")
print(f"\n공통 스타일 목록: {sorted(common_styles)}")

전체 고유 스타일 수: 27개
여성 전용 스타일: 19개
남성 전용 스타일: 4개
공통 스타일: 4개

공통 스타일 목록: ['hiphop', 'hippie', 'normcore', 'sportivecasual']
